In [ ]:
import mglyph as mg
import numpy as np

from mglyph_ml.dataset.export import create_dataset

def square(x: float, canvas: mg.Canvas):
    canvas.tr.scale(mg.lerp(x, 0.05, 0.95))
    canvas.rect(canvas.top_left, canvas.bottom_right, color="purple")

ds = create_dataset(name="experiment-1")
for x in np.concatenate([np.linspace(0.0, 40.0, 400), np.linspace(60.0, 100.0, 400)]):
    ds.add_sample(square, x, split="train")
for x in np.linspace(40.0, 60.0, 200):
    ds.add_sample(square, x, split="test")

ds.export(path='experiment-1.dataset')